# Register Meshes to the First Mesh of the Time-Series

This notebook transforms (i.e. registers, or aligns) the meshes of the time-series to the first mesh.

The meshes in the .ply files can be opened with:
- the `vscode-3d-preview` extension of VSCode
- [MeshLab](https://www.meshlab.net/)
- [Blender](https://www.blender.org/download/), specifically with its [Stop-motion-OBJ](https://github.com/neverhood311/Stop-motion-OBJ) plugin to visualize time-series of .ply files.

**Note:** Add a plugin to blender following [these instructions](https://docs.blender.org/manual/en/latest/editors/preferences/addons.html#rd-party-add-ons). 

## Set-up

In [1]:
import os
import subprocess

gitroot_path = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], universal_newlines=True
)
os.chdir(gitroot_path[:-1])
print("Working directory: ", os.getcwd())

import warnings

warnings.filterwarnings("ignore")

import sys

sys_dir = os.path.dirname(os.getcwd())
sys.path.append(sys_dir)
print("Directory added to path: ", sys_dir)

Working directory:  /home/nmiolane/code/my28brains
Directory added to path:  /home/nmiolane/code


## Imports

In [2]:
import glob

import numpy as np
import trimesh

import my28brains.io as io

MESHES_DIR = os.path.join(os.getcwd(), "data", "meshes")
REGISTERED_MESHES_DIR = os.path.join(os.getcwd(), "data", "registered_meshes")
print("MESHES_DIR: ", MESHES_DIR)
print("REGISTERED_MESHES_DIR: ", REGISTERED_MESHES_DIR)

MESHES_DIR:  /home/nmiolane/code/my28brains/data/meshes
REGISTERED_MESHES_DIR:  /home/nmiolane/code/my28brains/data/registered_meshes


## Functions to Register Meshes

In [3]:
def center_mesh(mesh):
    """Center a mesh by putting its barycenter at origin of the coordinates.

    Parameters
    ----------
    mesh : trimesh.Trimesh
        Mesh to center.

    Returns
    -------
    centered_mesh : trimesh.Trimesh
        Centered Mesh.
    """
    vertices = mesh.vertices
    centered_vertices = vertices - np.mean(vertices, axis=0)
    return trimesh.Trimesh(vertices=centered_vertices, faces=mesh.faces)


def register_mesh(mesh, base_mesh):
    """Register a mesh to a base mesh.

    Note that the rigid registration slightly un-centered the registered mesh.

    Parameters
    ----------
    mesh : trimesh.Trimesh
        Mesh to register.

    Returns
    -------
    registered_mesh : trimesh.Trimesh
        Registered Mesh.
    """
    transform_mesh_to_base_mesh, _ = trimesh.registration.mesh_other(
        mesh=mesh, other=base_mesh, scale=False
    )
    # Note: This modifies the original mesh in place
    registered_mesh = mesh.apply_transform(transform_mesh_to_base_mesh)
    return registered_mesh

## Left Hemisphere: Register Meshes on the First (Base) Mesh of the Time-Series

This takes a couple of minutes.

In [4]:
hemisphere = "left"
structure_id = -1
string_base = os.path.join(
    MESHES_DIR, f"{hemisphere}_structure_{structure_id}_sub-01_ses-**.ply"
)
paths = sorted(glob.glob(string_base))

print(
    f"Found {len(paths)} ply files for {hemisphere} hemisphere and anatomical structure {structure_id}:\n {paths}\n"
)

print(f"-> Registration: Using the first mesh as the base mesh.")
base_mesh = trimesh.load(paths[0])
centered_base_mesh = center_mesh(base_mesh)
ply_path = os.path.join(REGISTERED_MESHES_DIR, os.path.basename(paths[0]))
io.write_trimesh_to_ply(mesh=centered_base_mesh, ply_path=ply_path)

for path in paths[1:]:
    mesh = trimesh.load(path)
    centered_mesh = center_mesh(mesh)
    registered_mesh = register_mesh(mesh=centered_mesh, base_mesh=centered_base_mesh)

    ply_path = os.path.join(REGISTERED_MESHES_DIR, os.path.basename(path))
    io.write_trimesh_to_ply(mesh=registered_mesh, ply_path=ply_path)

Found 27 ply files for left hemisphere and anatomical structure -1:
 ['/home/nmiolane/code/my28brains/data/meshes/left_structure_-1_sub-01_ses-02.ply', '/home/nmiolane/code/my28brains/data/meshes/left_structure_-1_sub-01_ses-03.ply', '/home/nmiolane/code/my28brains/data/meshes/left_structure_-1_sub-01_ses-04.ply', '/home/nmiolane/code/my28brains/data/meshes/left_structure_-1_sub-01_ses-05.ply', '/home/nmiolane/code/my28brains/data/meshes/left_structure_-1_sub-01_ses-06.ply', '/home/nmiolane/code/my28brains/data/meshes/left_structure_-1_sub-01_ses-07.ply', '/home/nmiolane/code/my28brains/data/meshes/left_structure_-1_sub-01_ses-08.ply', '/home/nmiolane/code/my28brains/data/meshes/left_structure_-1_sub-01_ses-09.ply', '/home/nmiolane/code/my28brains/data/meshes/left_structure_-1_sub-01_ses-10.ply', '/home/nmiolane/code/my28brains/data/meshes/left_structure_-1_sub-01_ses-11.ply', '/home/nmiolane/code/my28brains/data/meshes/left_structure_-1_sub-01_ses-12.ply', '/home/nmiolane/code/my28bra

## Right Hemisphere: Register Meshes on the First (Base) Mesh of the Time-Series

This takes a couple of minutes.

In [5]:
hemisphere = "right"
structure_id = -1
string_base = os.path.join(
    MESHES_DIR, f"{hemisphere}_structure_{structure_id}_sub-01_ses-**.ply"
)
paths = sorted(glob.glob(string_base))

print(
    f"Found {len(paths)} ply files for {hemisphere} hemisphere and anatomical structure {structure_id}:\n {paths}\n"
)

print(f"-> Registration: Using the first mesh as the base mesh.")
base_mesh = trimesh.load(paths[0])
centered_base_mesh = center_mesh(base_mesh)
ply_path = os.path.join(REGISTERED_MESHES_DIR, os.path.basename(paths[0]))
io.write_trimesh_to_ply(mesh=centered_base_mesh, ply_path=ply_path)

for path in paths[1:]:
    mesh = trimesh.load(path)
    centered_mesh = center_mesh(mesh)
    registered_mesh = register_mesh(mesh=centered_mesh, base_mesh=centered_base_mesh)

    ply_path = os.path.join(REGISTERED_MESHES_DIR, os.path.basename(path))
    io.write_trimesh_to_ply(mesh=registered_mesh, ply_path=ply_path)

Found 25 ply files for right hemisphere and anatomical structure -1:
 ['/home/nmiolane/code/my28brains/data/meshes/right_structure_-1_sub-01_ses-02.ply', '/home/nmiolane/code/my28brains/data/meshes/right_structure_-1_sub-01_ses-03.ply', '/home/nmiolane/code/my28brains/data/meshes/right_structure_-1_sub-01_ses-04.ply', '/home/nmiolane/code/my28brains/data/meshes/right_structure_-1_sub-01_ses-06.ply', '/home/nmiolane/code/my28brains/data/meshes/right_structure_-1_sub-01_ses-07.ply', '/home/nmiolane/code/my28brains/data/meshes/right_structure_-1_sub-01_ses-08.ply', '/home/nmiolane/code/my28brains/data/meshes/right_structure_-1_sub-01_ses-10.ply', '/home/nmiolane/code/my28brains/data/meshes/right_structure_-1_sub-01_ses-11.ply', '/home/nmiolane/code/my28brains/data/meshes/right_structure_-1_sub-01_ses-12.ply', '/home/nmiolane/code/my28brains/data/meshes/right_structure_-1_sub-01_ses-14.ply', '/home/nmiolane/code/my28brains/data/meshes/right_structure_-1_sub-01_ses-16.ply', '/home/nmiolane/